In [10]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [11]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob2 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob3 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [12]:
import numpy as np

def generate_random_directed_graph(num_nodes, num_edges, cost_range=(1, 10)):
    # check the number of nodes and edges
    if num_edges < num_nodes or num_edges > num_nodes * (num_nodes - 1) // 2:
        raise ValueError("Number of edges must be at least num_nodes and no more than num_nodes * (num_nodes - 1) // 2.")

    # product nodes include the r
    nodes = ['r'] + [chr(i) for i in range(97, 97 + num_nodes - 1)]

    # create a cycle to ensure the strong connection
    cycle_edges = [(nodes[i], nodes[(i + 1) % num_nodes]) for i in range(num_nodes)]
    
    # produce other possible edge
    possible_edges = [(nodes[i], nodes[j]) for i in range(num_nodes) for j in range(num_nodes) if i != j and (nodes[j], nodes[i]) not in cycle_edges]

    # remove the duplicated edge
    np.random.shuffle(possible_edges)
    remaining_edges_count = num_edges - len(cycle_edges)
    selected_edges = cycle_edges + possible_edges[:remaining_edges_count]
    
    # produce edges with costs
    edges_with_costs = {edge: np.random.randint(cost_range[0], cost_range[1] + 1) for edge in selected_edges}

    return nodes, edges_with_costs

# 使用该函数生成一个随机有向图
num_nodes = 6  # 节点数量，包括 'r'
num_edges = 10  # 总边数，至少等于节点数
nodes, edges_with_costs = generate_random_directed_graph(num_nodes, num_edges)

print("Nodes:", nodes)
print("Directed edges with costs:", edges_with_costs)

Nodes: ['r', 'a', 'b', 'c', 'd', 'e']
Directed edges with costs: {('r', 'a'): 8, ('a', 'b'): 3, ('b', 'c'): 4, ('c', 'd'): 10, ('d', 'e'): 9, ('e', 'r'): 2, ('b', 'e'): 6, ('b', 'd'): 8, ('a', 'e'): 4, ('a', 'd'): 8}


In [118]:
'''
nodes = ['a','b','c','d','r']
edges_with_costs ={
    ('a','b'):3,('c','a'):1,('r','a'):9,('b','c'):8,
    ('b','r'):4,('d','b'):6,('r','c'):2,
    ('c','d'):7,('d','r'):5
}
'''

In [13]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)
y = pulp.LpVariable.dicts('y',(nodes,nodes),0,1,cat=pulp.LpInteger)
z = pulp.LpVariable.dicts('z',(nodes,nodes),0,1,cat=pulp.LpInteger)

In [14]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)
prob2 +=pulp.lpSum(edges_with_costs[u,v] * y[u][v] for u, v in edges_with_costs)
prob3 +=pulp.lpSum(edges_with_costs[u,v] * z[u][v] for u, v in edges_with_costs)

In [15]:
for v in nodes:
    if v != 'r':
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for u in nodes if (u,v) in edges_with_costs) >= 1

for u in nodes:
    if u != 'r':
        prob2 +=pulp.lpSum(y[u][v] for v in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for v in nodes if (u,v) in edges_with_costs) >= 1

In [16]:
prob += pulp.lpSum(x[u]['r'] for u in nodes if (u,'r') in edges_with_costs) == 0
prob += pulp.lpSum(x['r'][v] for v in nodes if ('r',v) in edges_with_costs)>= 1

prob2 += pulp.lpSum(y[u]['r'] for u in nodes if (u,'r') in edges_with_costs) >=1
prob2 += pulp.lpSum(y['r'][v] for v in nodes if ('r',v) in edges_with_costs) ==0

In [17]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if 'r' in s:
        subset.append(s)

In [18]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1
    summation2 = 0.0
    for u in sc:
        for v in s:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation2 += y[u][v]
    prob2 += summation2 >= 1
    
for s in subsets:
    sc = list(set(nodes) - set(s))  # complement of subset s
    summation3 = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs:  # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation3 += z[u][v]
    prob3 += summation3 >= 1  

In [19]:
prob.solve()
print ("Status:", LpStatus[prob.status])
prob2.solve()
print ("Status:", LpStatus[prob2.status])
prob3.solve()
print ("Status:", LpStatus[prob3.status])

Status: Optimal
Status: Optimal
Status: Optimal


In [20]:
print ("Optimal Solution for out")
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			print("(%s,%s)"%(u,v),)

            
print ("Optimal Solution for in")
for u in nodes:
	for v in nodes:
		if y[u][v].value() == 1:
			print("(%s,%s)"%(u,v),)

print ("Optimal Solution for strongly connected subgraph")
resultall = []
for u in nodes:
	for v in nodes:
		if z[u][v].value() == 1:
			resultall.append((u,v))
resultall

Optimal Solution for out
(r,a)
(a,b)
(a,d)
(a,e)
(b,c)
Optimal Solution for in
(a,b)
(b,c)
(c,d)
(d,e)
(e,r)
Optimal Solution for strongly connected subgraph


[('r', 'a'), ('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'e'), ('e', 'r')]

In [21]:
print ("Combine solution for out and in")
result=[]
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			result.append((u,v))
		elif y[u][v].value() == 1:
			result.append((u,v))
            
result

Combine solution for out and in


[('r', 'a'),
 ('a', 'b'),
 ('a', 'd'),
 ('a', 'e'),
 ('b', 'c'),
 ('c', 'd'),
 ('d', 'e'),
 ('e', 'r')]

In [136]:
print("Compare solution combine of out/in and for strongly connected subgraph")
print("result for strongly connected subgraph",resultall)
print("result for combine of out/in:", result)
print("Two result are same:", resultall==result)

Compare solution combine of out/in and for strongly connected subgraph
result for strongly connected subgraph [('a', 'b'), ('b', 'r'), ('c', 'a'), ('c', 'd'), ('d', 'r'), ('r', 'c')]
result for combine of out/in: [('a', 'b'), ('b', 'r'), ('c', 'a'), ('c', 'd'), ('d', 'r'), ('r', 'c')]
Two result are same: True
